# Citizen Developer Posting Board - Dash Application Wrapper

This notebook provides a central entry point for running the Dash-based Posting Board application.
It imports all necessary components and launches the web server.

**Important**: The Dash server will run in this notebook. You can access the web interface in your browser while the notebook is running.

## 1. Environment Setup and Path Configuration

In [ ]:
# Import system libraries
import os
import sys
import subprocess
import time
import warnings
from pathlib import Path

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Get current notebook directory
notebook_dir = Path.cwd()
print(f"Notebook directory: {notebook_dir}")

# Add backend directory to Python path
backend_path = notebook_dir / 'backend'
if backend_path.exists():
    sys.path.insert(0, str(backend_path))
    print(f"✓ Backend path added: {backend_path}")
else:
    print(f"✗ Backend directory not found at {backend_path}")
    print("  Please ensure you're running this notebook from the project root directory.")

## 2. Dependency Check and Installation

In [ ]:
# Check if required packages are installed
required_packages = {
    'dash': 'dash==2.14.1',
    'plotly': 'plotly==5.17.0',
    'flask': 'Flask==2.3.3',
    'flask_session': 'Flask-Session==0.5.0',
    'flask_cors': 'Flask-CORS==4.0.0',
    'sqlalchemy': 'SQLAlchemy==2.0.21',
    'pandas': 'pandas==2.0.3',
    'numpy': 'numpy==1.24.3',
    'werkzeug': 'werkzeug',
    'nest_asyncio': 'nest-asyncio'  # Required for Jupyter
}

missing_packages = []
for package, install_name in required_packages.items():
    try:
        __import__(package)
        print(f"✓ {package} is installed")
    except ImportError:
        print(f"✗ {package} is NOT installed")
        missing_packages.append(install_name)

if missing_packages:
    print("\n🔧 Installing missing packages...")
    for package in missing_packages:
        print(f"  Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package], 
                            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("✓ All packages installed successfully!")
else:
    print("\n✓ All required packages are already installed!")

## 3. Database Initialization and Verification

In [ ]:
# Change to backend directory for imports
os.chdir(backend_path)
print(f"Working directory: {os.getcwd()}")

# Import database components
try:
    from database import init_db, get_session
    from models import Idea, Skill, Claim, IdeaStatus, PriorityLevel
    
    # Initialize database
    print("\n🗄️  Initializing database...")
    init_db()
    
    # Verify database contents
    db = get_session()
    try:
        stats = {
            'Total Ideas': db.query(Idea).count(),
            'Open Ideas': db.query(Idea).filter(Idea.status == IdeaStatus.OPEN).count(),
            'Claimed Ideas': db.query(Idea).filter(Idea.status == IdeaStatus.CLAIMED).count(),
            'Completed Ideas': db.query(Idea).filter(Idea.status == IdeaStatus.COMPLETE).count(),
            'Total Skills': db.query(Skill).count(),
            'Total Claims': db.query(Claim).count()
        }
        
        print("\n📊 Database Statistics:")
        for key, value in stats.items():
            print(f"   {key}: {value}")
            
    finally:
        db.close()
        
    print("\n✓ Database initialized successfully!")
    
except Exception as e:
    print(f"\n❌ Database initialization error: {e}")
    print("   Please check your database configuration.")
    raise

## 4. Import and Configure Dash Application

In [ ]:
# Import nest_asyncio to handle event loop in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Import the Dash application
try:
    from dash_app import app, server
    
    print("✓ Dash application imported successfully!")
    print(f"\n📋 Application Details:")
    print(f"   Dash App: {app}")
    print(f"   Flask Server: {server}")
    print(f"   Secret Key Set: {'SECRET_KEY' in app.server.config}")
    
    # List available routes
    routes = [rule.rule for rule in server.url_map.iter_rules() if rule.rule.startswith('/')]
    print(f"\n🔗 Available Routes ({len(routes)} total):")
    for route in sorted(routes)[:10]:  # Show first 10
        if not route.startswith('/_'):
            print(f"   {route}")
    if len(routes) > 10:
        print(f"   ... and {len(routes) - 10} more")
        
except ImportError as e:
    print(f"\n❌ Import error: {e}")
    print("   Please ensure dash_app.py exists in the backend directory.")
    raise
except Exception as e:
    print(f"\n❌ Unexpected error: {e}")
    raise

## 5. Port Configuration and Availability Check

In [ ]:
import socket

def is_port_in_use(port, host='127.0.0.1'):
    """Check if a port is in use"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex((host, port)) == 0

def find_available_port(start_port=5000, max_attempts=10):
    """Find an available port starting from start_port"""
    for i in range(max_attempts):
        port = start_port + i
        if not is_port_in_use(port):
            return port
        print(f"   Port {port} is in use, checking {port + 1}...")
    raise RuntimeError(f"No available ports found between {start_port} and {start_port + max_attempts}")

# Find available port
print("🔍 Checking port availability...")
try:
    port = find_available_port(5000)
    print(f"\n✓ Found available port: {port}")
except RuntimeError as e:
    print(f"\n❌ {e}")
    print("   Please free up a port or adjust the port range.")
    raise

## 6. Launch Application (Blocking Mode)

**Choose ONE of the following methods to run the server:**

### Option A: Blocking Mode (Recommended for Development)
The server will run in this cell until you interrupt it (Kernel > Interrupt)

In [ ]:
# Run the Dash app in blocking mode
print(f"🚀 Starting Dash application on port {port}...\n")
print(f"📱 Access the application at: http://localhost:{port}")
print(f"🔐 Admin login: http://localhost:{port}/admin/login (password: 2929arch)")
print(f"\n⚠️  To stop the server: Use 'Kernel > Interrupt' or press the stop button")
print("━" * 60)

try:
    # Run the server
    app.run_server(
        host='0.0.0.0',
        port=port,
        debug=True,
        use_reloader=False,  # Disable reloader in Jupyter
        dev_tools_hot_reload=False  # Disable hot reload in Jupyter
    )
except KeyboardInterrupt:
    print("\n\n✓ Server stopped by user")
except Exception as e:
    print(f"\n\n❌ Server error: {e}")

## 7. Alternative: Background Mode

### Option B: Run Server in Background Thread
Use this if you want to continue using the notebook while the server runs

In [ ]:
import threading
import requests
from IPython.display import display, HTML

# Global variable to track server thread
server_thread = None

def run_server_background():
    """Run the Dash server in a background thread"""
    try:
        app.run_server(
            host='0.0.0.0',
            port=port,
            debug=False,  # Debug mode doesn't work well in threads
            use_reloader=False,
            threaded=True
        )
    except Exception as e:
        print(f"\n❌ Background server error: {e}")

# Stop existing server if running
if server_thread and server_thread.is_alive():
    print("⏹️  Stopping existing server...")
    # Note: There's no clean way to stop a Flask server in a thread
    print("   (Restart kernel to fully stop the server)")

# Start new server
server_thread = threading.Thread(target=run_server_background, daemon=True)
server_thread.start()

print(f"🚀 Starting server in background on port {port}...")
time.sleep(3)  # Give server time to start

# Verify server is running
try:
    response = requests.get(f"http://localhost:{port}/", timeout=5)
    if response.status_code == 200:
        print(f"\n✓ Server is running successfully!")
        display(HTML(f'''
        <div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; margin: 10px 0;">
            <h3 style="color: #2e7d32; margin: 0;">🎉 Dash Application is Running!</h3>
            <p style="margin: 10px 0;">Access your application at: 
                <a href="http://localhost:{port}" target="_blank" style="color: #1976d2; font-weight: bold;">
                    http://localhost:{port}
                </a>
            </p>
            <p style="margin: 10px 0;">Admin portal: 
                <a href="http://localhost:{port}/admin/login" target="_blank" style="color: #1976d2;">
                    http://localhost:{port}/admin/login
                </a> (password: 2929arch)
            </p>
        </div>
        '''))
except Exception as e:
    print(f"\n⚠️  Server may still be starting up or there was an error: {e}")
    print(f"   Try accessing http://localhost:{port} in your browser")

## 8. Server Management and Status Check

In [ ]:
# Check server status
def check_server_status():
    """Check if the server is responding"""
    try:
        response = requests.get(f"http://localhost:{port}/", timeout=2)
        if response.status_code == 200:
            print(f"✓ Server is running at http://localhost:{port}")
            print(f"  Status Code: {response.status_code}")
            print(f"  Response Size: {len(response.text)} bytes")
            
            # Check specific endpoints
            endpoints = ['/', '/submit', '/admin/login', '/my-ideas']
            print("\n  Endpoint Status:")
            for endpoint in endpoints:
                try:
                    r = requests.get(f"http://localhost:{port}{endpoint}", timeout=1)
                    print(f"    {endpoint}: {r.status_code}")
                except:
                    print(f"    {endpoint}: ❌ Not responding")
        else:
            print(f"⚠️  Server returned status code: {response.status_code}")
    except requests.exceptions.ConnectionError:
        print(f"❌ Server is not running on port {port}")
        print("   Please run one of the server launch cells above.")
    except Exception as e:
        print(f"❌ Error checking server: {e}")

check_server_status()

## 9. Quick Database Queries

Use these cells for database operations while the server is running:

In [ ]:
# View recent ideas with details
from database import get_session
from models import Idea, IdeaStatus, PriorityLevel

db = get_session()
try:
    recent_ideas = db.query(Idea).order_by(Idea.date_submitted.desc()).limit(5).all()
    
    print("📋 Recent Ideas:\n")
    for i, idea in enumerate(recent_ideas, 1):
        print(f"{i}. {idea.title}")
        print(f"   Status: {idea.status.value} | Priority: {idea.priority.value} | Size: {idea.size.value}")
        print(f"   Submitted: {idea.date_submitted.strftime('%Y-%m-%d %H:%M')}")
        print(f"   Skills: {', '.join([s.name for s in idea.skills]) if idea.skills else 'None'}")
        print(f"   By: {idea.submitter_name} ({idea.submitter_email})")
        print()
finally:
    db.close()

In [ ]:
# Add a new skill
def add_skill(name, description=""):
    """Add a new skill to the database"""
    from database import get_session
    from models import Skill
    
    db = get_session()
    try:
        existing = db.query(Skill).filter(Skill.name == name).first()
        if existing:
            print(f"⚠️  Skill '{name}' already exists")
            return existing
        
        skill = Skill(name=name, description=description)
        db.add(skill)
        db.commit()
        print(f"✓ Added skill: {name}")
        return skill
    except Exception as e:
        db.rollback()
        print(f"❌ Error adding skill: {e}")
    finally:
        db.close()

# Example: Add a new skill
# add_skill("Machine Learning", "Experience with ML frameworks and algorithms")

In [ ]:
# Dashboard statistics
from database import get_session
from models import Idea, Skill, Claim, IdeaStatus, PriorityLevel
from sqlalchemy import func
from datetime import datetime, timedelta

db = get_session()
try:
    # Overall statistics
    total_ideas = db.query(Idea).count()
    open_ideas = db.query(Idea).filter(Idea.status == IdeaStatus.OPEN).count()
    claimed_ideas = db.query(Idea).filter(Idea.status == IdeaStatus.CLAIMED).count()
    completed_ideas = db.query(Idea).filter(Idea.status == IdeaStatus.COMPLETE).count()
    
    # Recent activity (last 7 days)
    week_ago = datetime.now() - timedelta(days=7)
    recent_ideas = db.query(Idea).filter(Idea.date_submitted >= week_ago).count()
    recent_claims = db.query(Claim).filter(Claim.claim_date >= week_ago).count()
    
    # Priority breakdown
    priority_counts = db.query(
        Idea.priority, func.count(Idea.id)
    ).filter(
        Idea.status == IdeaStatus.OPEN
    ).group_by(Idea.priority).all()
    
    print("📊 Dashboard Statistics\n")
    print("Overall:")
    print(f"  Total Ideas: {total_ideas}")
    print(f"  Open: {open_ideas} ({open_ideas/total_ideas*100:.1f}%)" if total_ideas > 0 else "  Open: 0")
    print(f"  Claimed: {claimed_ideas} ({claimed_ideas/total_ideas*100:.1f}%)" if total_ideas > 0 else "  Claimed: 0")
    print(f"  Completed: {completed_ideas} ({completed_ideas/total_ideas*100:.1f}%)" if total_ideas > 0 else "  Completed: 0")
    
    print("\nLast 7 Days:")
    print(f"  New Ideas: {recent_ideas}")
    print(f"  New Claims: {recent_claims}")
    
    print("\nOpen Ideas by Priority:")
    for priority, count in priority_counts:
        print(f"  {priority.value}: {count}")
        
finally:
    db.close()

## 10. Development Tools

Useful functions for development and debugging:

In [ ]:
# Create sample data for testing
def create_sample_idea(title, description, priority="medium", skills=None):
    """Create a sample idea for testing"""
    from database import get_session
    from models import Idea, Skill, IdeaStatus, PriorityLevel, IdeaSize
    from datetime import datetime
    
    db = get_session()
    try:
        idea = Idea(
            title=title,
            description=description,
            submitter_name="Test User",
            submitter_email="test@example.com",
            team_name="Test Team",
            priority=PriorityLevel(priority),
            size=IdeaSize.MEDIUM,
            status=IdeaStatus.OPEN,
            date_submitted=datetime.now(),
            target_date=datetime.now() + timedelta(days=30)
        )
        
        # Add skills if provided
        if skills:
            for skill_name in skills:
                skill = db.query(Skill).filter(Skill.name == skill_name).first()
                if skill:
                    idea.skills.append(skill)
        
        db.add(idea)
        db.commit()
        print(f"✓ Created idea: {title} (ID: {idea.id})")
        return idea
    except Exception as e:
        db.rollback()
        print(f"❌ Error creating idea: {e}")
    finally:
        db.close()

# Example usage:
# create_sample_idea(
#     "Automate Report Generation", 
#     "Create a tool to automatically generate weekly reports",
#     priority="high",
#     skills=["Python", "Data Analysis"]
# )

In [ ]:
# Export data to CSV
def export_ideas_to_csv(filename="ideas_export.csv"):
    """Export all ideas to a CSV file"""
    import pandas as pd
    from database import get_session
    from models import Idea
    
    db = get_session()
    try:
        ideas = db.query(Idea).all()
        
        data = []
        for idea in ideas:
            data.append({
                'ID': idea.id,
                'Title': idea.title,
                'Description': idea.description[:100] + '...' if len(idea.description) > 100 else idea.description,
                'Status': idea.status.value,
                'Priority': idea.priority.value,
                'Size': idea.size.value,
                'Submitter': idea.submitter_name,
                'Team': idea.team_name,
                'Date Submitted': idea.date_submitted.strftime('%Y-%m-%d'),
                'Skills': ', '.join([s.name for s in idea.skills])
            })
        
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"✓ Exported {len(data)} ideas to {filename}")
        return df
    finally:
        db.close()

# Example usage:
# export_ideas_to_csv()

## 11. Shutdown and Cleanup

To stop the server:
1. **If running in blocking mode**: Use `Kernel > Interrupt` or click the stop button
2. **If running in background**: Use `Kernel > Restart` to fully stop all threads

The database file (`posting_board.db`) persists between sessions.

In [ ]:
# Optional: Clean shutdown message
print("\n🛑 To stop the server:")
print("   - Blocking mode: Kernel > Interrupt")
print("   - Background mode: Kernel > Restart")
print("\n💾 Your data is saved in: backend/posting_board.db")